In [ ]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import os 


#기업코드를 입력받아 기사의 제목을 100페이지까지 크롤링하는 함수

def crawler(company_code, maxpage):
    
    page = 1 
    all_news= pd.DataFrame(columns={'날짜','언론사','기사제목','링크'})
    while page <= int(maxpage): 
        
        url = 'https://finance.naver.com/item/news_news.nhn?code=' + str(company_code) + '&page=' + str(page) 
        source_code = requests.get(url).text
        html = BeautifulSoup(source_code, "lxml")

        # 뉴스 제목 
        titles = html.select('.title')
        title_result=[]
        for title in titles: 
            title = title.get_text() 
            title = re.sub('\n','',title)
            title_result.append(title)
 
 
        # 뉴스 링크
        links = html.select('.title') 
 
        link_result =[]
        for link in links: 
            add = 'https://finance.naver.com' + link.find('a')['href']
            link_result.append(add)
 
 
        # 뉴스 날짜 
        dates = html.select('.date') 
        date_result = [date.get_text() for date in dates] 

 
        # 뉴스 매체     
        sources = html.select('.info')
        source_result = [source.get_text() for source in sources] 
 
 
        # 변수들 합쳐서 해당 디렉토리에 csv파일로 저장하기 
        
        result= {"날짜" : date_result, "언론사" : source_result, "기사제목" : title_result, "링크" : link_result} 
        df_result = pd.DataFrame(result)
        
        all_news=pd.concat([all_news, df_result])
        
        print("다운 받고 있습니다------")
        page += 1 
    all_news=all_news.drop_duplicates(['기사제목'])
    all_news.to_csv(str(company_code) + '.csv', mode='w', encoding='utf-8-sig') 
    print('all_news')
    print(all_news)

#train 데이터를 크롤링
train_df=pd.read_csv('train종목코드.csv')
train_df['종목코드']=train_df['CODE'].astype('str').str.zfill(6)
for i in train_df['종목코드']:
    crawler(i,100)

#test 데이터를 크롤링
train_df=pd.read_csv('train종목코드.csv')
train_df['종목코드']=train_df['CODE'].astype('str').str.zfill(6)
for i in train_df['종목코드']:
    crawler(i,100)

In [ ]:
def neg_rate(df):
    
    e_rates=[]
    s_rates=[]
    g_rates=[]
    
    for i in df['CODE']:
        path=f'{i}.csv'
        news= pd.read_csv(path)

        e_count=0
        s_count=0
        g_count=0
        
        for t in news['기사제목']:
            e_list=['환경']
            s_list=['담합','불법파견','불매운동','불법고용','불공정거래','하도급문제','논란','골목상권 위협']
            g_list=['갑질','비리','뇌물','분식회계','비자금','일감몰아주기','배임','횡령','탈세','밀어내기','차명계좌','주가조작','성과급 잔치','구속','조세회피','내부자거래']
            
            if '환경' in t:
                e_count += 1
            elif ('담합'in t or '불법파견'in t or'불매운동' in t or'불법고용' in t or '불공정거래' in t or '하도급문제' in t or '논란'in t or '골목상권 위협' in t):
                s_count+=1
            elif ('갑질' in t or '비리'in t or '뇌물'in t or '분식회계' in t or '비자금' in t or '일감몰아주기' in t or '배임' in t or '횡령' in t or '탈세' in t or '밀어내기' in t or '차명계좌' in t or '주가조작' in t or '성과급 잔치' in t or '구속' in t or '조세회피' in t or '내부자거래'in t):
                g_count+=1
            e_rate= e_count / len(news['기사제목'])
            s_rate= s_count / len(news['기사제목'])
            g_rate= g_count / len(news['기사제목'])
            
        e_rates.append(e_rate)
        s_rates.append(s_rate)
        g_rates.append(g_rate)
            
    df['e 긍정뉴스 비율'] = e_rates
    df['s 부정뉴스 비율'] = s_rates
    df['g 부정뉴스 비율'] = g_rates
    
    news_df = df
    news_df.to_csv('train.csv',encoding="utf-8-sig")
    return news_df